# Preprocess and window longitudinal datasets and save afterwards

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.insert(0, '/home/jovyan/braindecode/')
import pickle

In [3]:
from decode_tueg import (
    get_longitudinal_ds, preprocess, get_preprocessors,
    _create_windows, _create_final_scores, load_exp,
    get_n_preds_per_input, DataScaler, TargetScaler,
    Augmenter, iter_exp_dir,
)

Tue Feb 14 15:16:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:3E:00.0 Off |                  N/A |
| 27%   26C    P8     9W / 250W |      1MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
exp_date = '2023-02-09T09:34:21.685021'
exp_dir = '/home/jovyan/experiments/'
mapping = None  # where to get this from?
checkpoint = 'train_end'

In [5]:
for x in iter_exp_dir(exp_dir, exp_date):
    break

In [6]:
estimator, data_scaler, target_scaler, config = load_exp(
    exp_dir, os.path.join(exp_dir, x), checkpoint)

In [7]:
min_age = int(config.min_age)
max_age = int(config.max_age)
tmin = int(config.tmin)
tmax = int(config.tmax)
n_jobs = int(config.n_jobs)
window_size_samples = int(config.window_size_samples)
preload = int(config.preload)
target_name = config.target_name
out_dir = config.out_dir

In [12]:
#'pathological', 'non_pathological', 'transition'
for ds_name in ['pathological', 'non_pathological', 'transition']:
    print(f'getting {ds_name}')
    ds = get_longitudinal_ds(ds_name, (min_age, max_age))
    print('preprocessing')
    ds = preprocess(
        ds, 
        preprocessors=get_preprocessors(tmin, tmax), 
        n_jobs=n_jobs,
    )
    n_preds_per_input = get_n_preds_per_input(
        estimator.module, ds[0][0].shape[0], window_size_samples)
    print('windowing')
    ds = _create_windows(
        ds,
        window_size_samples,
        n_jobs, 
        preload,
        n_preds_per_input,
        mapping,
    )
    with open(f'/home/jovyan/longitudinal/{ds_name}_pre_win.pkl', 'wb') as f:
        pickle.dump(ds, f)

getting
preprocessing
2023-02-14 12:24:39 DEBUG    adding cropper 60 – None
2023-02-14 12:26:26 DEBUG    model produces 5504 preds for every input of size 6000
windowing
getting
preprocessing
2023-02-14 13:09:17 DEBUG    adding cropper 60 – None
2023-02-14 13:09:37 DEBUG    model produces 5504 preds for every input of size 6000
windowing
getting
preprocessing
2023-02-14 13:15:41 DEBUG    adding cropper 60 – None
2023-02-14 13:15:58 DEBUG    model produces 5504 preds for every input of size 6000
windowing


In [13]:
print("hiu")

hiu


# Predict the longitudinal datasets

In [1]:
import os
import sys
sys.path.insert(0, '/home/jovyan/braindecode/')
import pickle

In [2]:
import numpy as np
import pandas as pd

In [3]:
from decode_tueg import (
    deconfound, load_exp, _create_final_scores, plot_age_gap_hist, plot_heatmaps,
    save_csv, plot_age_gap_hist_and_permutation_test, DataScaler, TargetScaler,
    Augmenter, iter_exp_dir,
)

Tue Feb 14 15:17:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:3E:00.0 Off |                  N/A |
| 27%   29C    P8    10W / 250W |      1MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
exp_dir = '/home/jovyan/experiments/'
exp_date = '2023-02-09T09:34:21.685021'
checkpoint = 'train_end'

In [7]:
n_recordings = 10

In [8]:
# read all experiment models (seeds and cv / eval runs) to predict longitudinal datasets
# 'transition', 'non_pathological', 'pathological'
for ds_name in ['transition', 'non_pathological', 'pathological']:
    for exp_path in iter_exp_dir(exp_dir, exp_date):
        print(f'loading exp {exp_path}')
        estimator, data_scaler, target_scaler, config = load_exp(
            exp_dir, exp_path, checkpoint)
        print(f'getting {ds_name}')
        with open(f'/home/jovyan/longitudinal/{ds_name}_pre_win.pkl', 'rb') as f:
            ds = pickle.load(f)
        if n_recordings not in [-1, None]:
            ds = ds.split(list(range(n_recordings)))['0']
        print('predicting')
        ds_preds, ds_score = _create_final_scores(
            estimator,
            ds,
            ds_name,
            config.target_name,
            target_scaler,
            data_scaler,
            int(config.n_jobs),
            subject_wise=True,
        )
        save_path = os.path.join(exp_dir, exp_path, 'preds')
        print(f'saving {save_path}')
        # write preds to file
        #save_csv(
        #    ds_preds,
        #    save_path,
        #    f'{checkpoint}_longitudinal_{ds_name}_preds.csv',
        #)
        # write scores to file
        scores = pd.read_csv(
            os.path.join(os.path.join(exp_dir, exp_path), f'{checkpoint}_scores.csv'), 
            index_col=0,
        )
        for k, v in ds_score.items():
            scores[k] = v
        #save_csv(
        #    scores,
        #    os.path.join(exp_dir, exp_path),
        #    f'{checkpoint}_scores.csv',
        #)

loading exp 2023-02-09T09:34:21.685021/20230206/0
getting transition
predicting
2023-02-14 15:18:14 INFO     on transition reached
2023-02-14 15:18:18 INFO     7.56 mae
2023-02-14 15:18:18 INFO     7.56 mdae
2023-02-14 15:18:18 INFO     -16.07 r2
2023-02-14 15:18:18 INFO     0.16 mape
2023-02-14 15:18:18 INFO     0.16 mdape
saving /home/jovyan/experiments/2023-02-09T09:34:21.685021/20230206/0/preds
loading exp 2023-02-09T09:34:21.685021/20230207/0
getting transition
predicting
2023-02-14 15:18:22 INFO     on transition reached
2023-02-14 15:18:24 INFO     7.90 mae
2023-02-14 15:18:24 INFO     7.90 mdae
2023-02-14 15:18:24 INFO     -17.88 r2
2023-02-14 15:18:24 INFO     0.17 mape
2023-02-14 15:18:24 INFO     0.17 mdape
saving /home/jovyan/experiments/2023-02-09T09:34:21.685021/20230207/0/preds
loading exp 2023-02-09T09:34:21.685021/20230208/0
getting transition
predicting
2023-02-14 15:18:28 INFO     on transition reached
2023-02-14 15:18:31 INFO     4.83 mae
2023-02-14 15:18:31 INFO   

In [11]:
print("hi")

hi


# Read the longitudinal predictions and plot them

In [12]:
import os
import sys
sys.path.insert(0, '/home/jovyan/braindecode/')

In [13]:
import pandas as pd

In [14]:
from decode_tueg import (
    deconfound, plot_age_gap_hist, plot_heatmaps,
    plot_age_gap_hist_and_permutation_test, iter_exp_dir,
)

In [15]:
exp_dir = '/home/jovyan/experiments/'
exp_date = '2023-02-09T09:34:21.685021'
checkpoint = 'train_end'

In [17]:
bin_width = 2
n_repetitions = 1000  # actually 100000!
#detrend = 'quadratic'

In [18]:
from decode_tueg import deconfound

In [ ]:
cols = ['y_true', 'y_pred', 'subject', 'pathological']
for ds_name in ['transition', 'non_pathological']:#, 'pathological']:
    all_preds, configs = [], []
    for i, exp_path in enumerate(iter_exp_dir(exp_dir, exp_date)):
        preds_path = os.path.join(exp_dir, exp_path, 'preds', f'{checkpoint}_longitudinal_{ds_name}_preds.csv')
        preds = pd.read_csv(preds_path, index_col=0)[cols]
        all_preds.append(preds)
    all_preds = pd.concat(all_preds)
    all_preds = all_preds.reset_index().groupby('index').mean().reset_index(drop=True)

    plot_heatmaps(all_preds, bin_width)
    # possible to deconfound here?
    #all_preds = deconfound(all_preds, detrend)
    if ds_name == 'transition':
        plot_age_gap_hist_and_permutation_test(all_preds, bin_width, n_repetitions)
    else:
        plot_age_gap_hist(all_preds, bin_width)

In [15]:
from decode_tueg import reject_derivating_ages

In [16]:
for ds_i, ds_ in enumerate(ds.datasets):
    break

In [18]:
_ = [ds_.set_description(
    {'path': ds_.description['path'].replace(
        '/data/datasets/TUH/EEG/tuh_eeg', '/home/jovyan/mne_data/TUH/tuh_eeg/')},
    overwrite=True,
) for ds_i, ds_ in enumerate(ds.datasets)]